In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Input, Conv2D, BatchNormalization
from keras.layers import MaxPool2D, MaxPooling2D, Reshape, Dropout, SeparableConv2D
from keras.models import Model
from keras.utils import to_categorical
import os

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline
from WavDataLoader import WavDataLoader
from WavDataGenerator import WavDataGenerator

Using TensorFlow backend.


In [7]:
nx = 128
ny = 32
labels = ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']
num_labels = len(labels)

In [8]:
if os.name is 'nt':
    data_dir = r'C:\Development\kaggle\tensorflow-speech-recognition-challenge\data\train\audio'
else:
    data_dir = r'/home/shaur141/Development/kaggle/tensorflow-speech-recognition-challenge/data/train/audio'
    
# wav_data_generator = WavDataGenerator(data_dir, labels, nx=nx, ny=ny, is_train=True, batch_size=32)
wav_data_loader = WavDataLoader(data_dir, labels, nx=nx, ny=ny)


In [17]:
def build_model():
    inputs = Input(shape=(nx, ny,1))
#     x = Reshape((wav_data_loader.nx*wav_data_loader.ny,))(inputs)
#     x = BatchNormalization()(inputs)
    x = Conv2D(16,(3,3),strides=(1,1), activation='relu')(inputs)
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32,(3,3),strides=(2,2), activation='relu')(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64,(3,3),strides=(2,2), activation='relu')(x)    
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = SeparableConv2D(128,(3,3),strides=(2,2), activation='relu')(x) 
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)       
    x = Reshape((-1,))(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
#     x = Dense(128, activation='relu')(x)
    predictions = Dense(num_labels, activation='softmax')(x)    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model    

In [18]:
model = build_model()

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 30, 16)       160       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 125, 29, 16)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 125, 29, 16)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 62, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 61, 13, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 61, 13, 32)        0         
__________

In [20]:
model.fit(x=wav_data_loader.X, y=to_categorical(wav_data_loader.y), validation_split=0.15, epochs=20)

# model.fit_generator(wav_data_generator.generator(), 
#                     steps_per_epoch=wav_data_generator.num_examples//wav_data_generator.batch_size,
#                    workers=4)
                    

Train on 56825 samples, validate on 10028 samples
Epoch 1/20
56825/56825 [==============================] - 37s - loss: 1.6937 - acc: 0.4967 - val_loss: 0.8307 - val_acc: 0.7514
Epoch 2/20
56825/56825 [==============================] - 36s - loss: 0.8097 - acc: 0.7544 - val_loss: 0.5595 - val_acc: 0.8291
Epoch 3/20
56825/56825 [==============================] - 37s - loss: 0.6379 - acc: 0.8080 - val_loss: 0.4503 - val_acc: 0.8630
Epoch 4/20
56825/56825 [==============================] - 37s - loss: 0.5625 - acc: 0.8300 - val_loss: 0.4330 - val_acc: 0.8713
Epoch 5/20
56825/56825 [==============================] - 37s - loss: 0.5152 - acc: 0.8443 - val_loss: 0.4160 - val_acc: 0.8759
Epoch 6/20
56825/56825 [==============================] - 37s - loss: 0.4859 - acc: 0.8522 - val_loss: 0.3872 - val_acc: 0.8837
Epoch 7/20
56825/56825 [==============================] - 37s - loss: 0.4724 - acc: 0.8582 - val_loss: 0.3964 - val_acc: 0.8824
Epoch 8/20
56825/56825 [==============================

In [21]:
model.save('model.h5')